In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# 그래프에서 한글 깨짐 방지
import platform
if platform.system() == "Darwin":          # MAC인 경우
    plt.rc('font', family ="AppleGothic")
else:
    plt.rc('font',family='Malgun Gothic')

#멧플로립에서 음수를 표시
plt.rc('axes',unicode_minus=False)

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')

In [19]:
df=pd.read_csv('../외감기업/raw_data/t-1_raw_외감.csv',encoding='euc-kr')

In [21]:
df['자산총계(천원)'] = df['자산총계(천원)'] * 1000
df['log자산총계'] = np.log(df['자산총계(천원)'])
df['잉여현금흐름(FCF)']=df['OCF(천원)']-df['ICF(천원)']
df['전체현금흐름(천원)']=df['OCF(천원)']+df['ICF(천원)']+df['FCF(천원)']

df['총자산대비잉여현금흐름'] = df['잉여현금흐름(FCF)']/df['자산총계(천원)']
df['총자산대비현금흐름'] = df['전체현금흐름(천원)'] / df['자산총계(천원)']
df=df[['회사명', '거래소코드','유동자산회전률', '총자산대비잉여현금흐름','자기자본구성비율', 'log자산총계','자기자본회전률', '순운전자본회전률', '자기자본증가율', '총자본증가율', '총자산대비현금흐름', '총자본투자효율','t-1감사의견코드']]

c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


---
- 결측치, 무한대값 삭제

In [26]:
df.isna().sum()
df.dropna(axis=0,inplace=True)

In [30]:
df.isin([np.inf,-np.inf]).sum()
df = df.replace([np.inf, -np.inf], np.nan)  # inf 값을 NaN으로 대체
df = df.dropna()  # NaN이 있는 행 삭제

In [35]:
# df.iloc[:,2:12]

In [37]:
scaler=MinMaxScaler()
df.iloc[:,2:12]=scaler.fit_transform(df.iloc[:,2:12])

In [39]:
# dp-Score의 계수
coefficients = {
    '유동자산회전률':np.exp(0.47765086),                    # 높아야 좋다
    '총자산대비잉여현금흐름':np.exp(0.53785436),            # 높아야 좋다
    '자기자본구성비율':np.exp(0.49360789),                  # 높아야 좋다
    'log자산총계':np.exp(0.94238478),                      # 높아야 좋다
    '자기자본회전률':np.exp(-0.08646556),                   # 높아야 좋다
    '순운전자본회전률':np.exp(-0.98107367),                 # 높아야 좋다
    '자기자본증가율':np.exp(0.19544705),                    # 높아야 좋다
    '총자본증가율':np.exp(0.44094258),                      # 높아야 좋다
    '총자산대비현금흐름':np.exp(-0.37036301),               # 높아야 좋다
    '총자본투자효율':np.exp(0.26458857),                    # 높아야 좋다

    'constant': 0.90495308
}


# O-Score 계산
df['dp_score'] = coefficients['constant']
df['dp_score'] += coefficients['유동자산회전률'] * df['유동자산회전률']
df['dp_score'] += coefficients['총자산대비잉여현금흐름'] * df['총자산대비잉여현금흐름']
df['dp_score'] += coefficients['자기자본구성비율'] * df['자기자본구성비율']
df['dp_score'] += coefficients['log자산총계'] * df['log자산총계']
df['dp_score'] += coefficients['자기자본회전률'] * df['자기자본회전률']
df['dp_score'] += coefficients['순운전자본회전률'] * df['순운전자본회전률']
df['dp_score'] += coefficients['자기자본증가율'] * df['자기자본증가율']
df['dp_score'] += coefficients['총자본증가율'] * df['총자본증가율']
df['dp_score'] += coefficients['총자산대비현금흐름'] * df['총자산대비현금흐름']
df['dp_score'] += coefficients['총자본투자효율'] * df['총자본투자효율']

# 로지스틱 함수를 이용해 부도 확률 계산
# df_등급화['default_probability'] = 1 / (1 + np.exp(-df_등급화['dp_score']))

In [40]:
df.shape

(188632, 14)

In [41]:
df.describe()

,거래소코드,유동자산회전률,총자산대비잉여현금흐름,자기자본구성비율,log자산총계,자기자본회전률,순운전자본회전률,자기자본증가율,총자본증가율,총자산대비현금흐름,총자본투자효율,t-1감사의견코드,dp_score
count,188632.000000,188632.000000,188632.000000,188632.000000,188632.000000,188632.000000,188632.000000,188632.000000,188632.000000,188632.000000,188632.000000,188632.000000,188632.000000
mean,57668.828921,0.001589,0.000890,0.471247,0.572036,0.000012,0.000157,0.000436,0.000218,0.684553,0.663496,0.071430,4.487016
std,25364.722797,0.002318,0.002301,0.215488,0.058218,0.002375,0.002325,0.003264,0.002306,0.013657,0.002070,0.257543,0.407585
min,10000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.285626
25%,41837.000000,0.001578,0.000877,0.288338,0.533038,0.000002,0.000132,0.000408,0.000190,0.681765,0.663478,0.000000,4.159155
50%,59713.000000,0.001580,0.000882,0.445937,0.559802,0.000002,0.000143,0.000409,0.000195,0.683397,0.663484,0.000000,4.452505
75%,76679.000000,0.001583,0.000891,0.633326,0.599735,0.000004,0.000155,0.000412,0.000215,0.686720,0.663493,0.000000,4.788795
max,107278.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.199626


In [42]:
df[df['t-1감사의견코드']==1]['dp_score'].mean()

4.2188499153072385

- dp_score
- 신용평점	
- A	    67.025568
- AA	69.826705
- B 	53.369724
- BB	57.360487
- BBB	60.007862
- CC	42.225740
- CCC	49.269174
- D	30.959408